In [1]:
%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit import QuantumRegister, ClassicalRegister
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.circuit import Parameter

import qiskit.ignis.verification.tomography as tom
from qiskit.providers.ibmq.managed import IBMQJobManager
from qiskit.providers.jobstatus import JOB_FINAL_STATES
from qiskit.quantum_info import state_fidelity
from qiskit.providers.aer import noise

# Importing other packages
import pickle
import numpy as np

# Loading your IBM Q account(s)
provider = IBMQ.load_account()

In [2]:
from copy import deepcopy

In [3]:
print('\n'.join([str(x) for x in provider.backends()]))
print('---')
print('\n'.join([str(x) for x in Aer.backends()]))
backend=provider.get_backend('ibmq_16_melbourne')
backend_sim=provider.get_backend('ibmq_qasm_simulator')
backend_sv=Aer.get_backend('statevector_simulator')

ibmq_qasm_simulator
ibmqx2
ibmq_16_melbourne
ibmq_vigo
ibmq_ourense
ibmq_london
ibmq_burlington
ibmq_essex
ibmq_armonk
ibmq_athens
ibmq_rome
---
qasm_simulator
statevector_simulator
unitary_simulator
pulse_simulator


In [4]:
data_list=[0,1,2,3,4]
syn=[5]
flag=[6]

In [5]:
def StatePreparation(stabilizer=0):
    q = QuantumRegister(7)
    qc = QuantumCircuit(q)
    #prepare the 5-qubit graph state
    qc.h(q[0])
    qc.h(q[1])
    qc.h(q[2])
    qc.h(q[3])
    qc.h(q[4])
    qc.cz(q[0],q[1])
    qc.cz(q[2],q[3])
    qc.cz(q[1],q[2])
    qc.cz(q[3],q[4])
    qc.cz(q[0],q[4])
    #syndrome and flag extraction
    #stabilizer order: from left to right, Fig 2(a), SM of two extra qubits
    if(stabilizer==0):
        #first stabilizer 
        qc.h(q[5])
        qc.cx(q[5],q[0])
        qc.cx(q[5],q[6])
        qc.cz(q[1],q[5])
        qc.cx(q[5],q[6])
        qc.cz(q[4],q[5])
        qc.h(q[5])
    elif(stabilizer==1):
    #second stabilizer
        qc.h(q[5])
        qc.cz(q[0],q[5])
        qc.cx(q[5],q[6])
        qc.cx(q[5],q[1])
        qc.cx(q[5],q[6])
        qc.cz(q[2],q[5])
        qc.h(q[5])
    elif(stabilizer==2):
        #third stabilizer
        qc.h(q[5])
        qc.cx(q[5],q[3])
        qc.cx(q[5],q[6])
        qc.cz(q[2],q[5])
        qc.cx(q[5],q[6])
        qc.cz(q[4],q[5])
        qc.h(q[5])
    #measure syndrome and flag & do state tomography for the data qubits
    qc.barrier()
    #qc.measure(q[5],c[5])
    #qc.measure(q[6],c[6])
    return q, qc

In [6]:
q, qc=StatePreparation(0)
qc.draw()

┌───┐         ┌───┐                      ░ 
q0_0: ┤ H ├─■─────■─┤ X ├──────────────────────░─
      ├───┤ │     │ └─┬─┘                      ░ 
q0_1: ┤ H ├─■──■──┼───┼────────■───────────────░─
      ├───┤    │  │   │        │               ░ 
q0_2: ┤ H ├─■──■──┼───┼────────┼───────────────░─
      ├───┤ │     │   │        │               ░ 
q0_3: ┤ H ├─■──■──┼───┼────────┼───────────────░─
      ├───┤    │  │   │        │               ░ 
q0_4: ┤ H ├────■──■───┼────────┼───────■───────░─
      ├───┤           │        │       │ ┌───┐ ░ 
q0_5: ┤ H ├───────────■────■───■───■───■─┤ H ├─░─
      └───┘              ┌─┴─┐   ┌─┴─┐   └───┘ ░ 
q0_6: ───────────────────┤ X ├───┤ X ├─────────░─
                         └───┘   └───┘         ░

To keep the consistence, the following blocks are mainly from `test state prep.ipynb`

In [7]:
import time

Based on https://github.com/Qiskit/qiskit-tutorials/blob/master/legacy_tutorials/ignis/6a_state_tomography.ipynb?fbclid=IwAR1Pf1nH2CBPUWDdsa_xBiYs1kYDu8hwI9dVvr63z7ypmhy2UlR6QpMgJWc, 2-Qubit Conditional State Tomography

we should have projected the ideal final state to the final state, 'reduced(v,sf)' returns the projected states based on 'sf'. At the moment, for the simulation result, 'sf' can only be '00' and the func returns the first 32 elements of v.

In [8]:
def reduced(v, sf):
    s=int(sf[0])
    f=int(sf[1])
    out=s*2+f
    res=[]
    for i in range(v.shape[0]):
        if((i>>5)==out):#((i%4)==out):#
            res.append(v[i])
    res=np.array(res)
    res=res/np.sqrt(np.sum(np.power(np.abs(res),2)))
    return np.array(res)

## Main part

Circuit | Description 
---| ---
qc0 | As shown in [6], state prepartion + stabilizer, no measurement
qc1 | add measurements on syndrome and flag qubits
qctom | add measurements on syndrome and flag qubits, then do state tomography on data qubits, used for tomography by both simulation and experiment
qctom_no_anc | qctom without measurement, used for 'tom.stateTomographyFitter'


In [10]:
def executeStatePreparation(stabilizer=0):
    #state vector
    q0, qc0=StatePreparation(stabilizer)
    qc0.draw()
    
    qc1=deepcopy(qc0)
    ca1 = ClassicalRegister(2)
    qc1.add_register(ca1)
    qc1.measure([q0[5],q0[6]],[ca1[0],ca1[1]])
    job = execute(qc1, backend_sv)
    psi_qc = job.result().get_statevector(qc1)
    #qasm simulation
    qctom = tom.state_tomography_circuits(qc0, data_list, meas_labels='Pauli', meas_basis='Pauli')
    qctom_no_anc = deepcopy(qctom)
    for qc in qctom:
        ca = ClassicalRegister(2)
        qc.add_register(ca)
        qc.measure([q0[5],q0[6]],[ca[0],ca[1]])
        
    job = execute(qctom, backend=backend_sim, optimization_level=0, shots=4096)
    results_sim=job.result()

    #strip the registers for syndrome and flag extraction
    result_sf={}
    for sf in ['00','01','10','11']:
            #outcomes: 00,01,10,11
            new_result = deepcopy(results_sim)

            for resultidx, _ in enumerate(results_sim.results):
                old_counts = results_sim.get_counts(resultidx)
                new_counts = {}

                #change the size of the classical register
                new_result.results[resultidx].header.creg_sizes = [new_result.results[resultidx].header.creg_sizes[0]]
                new_result.results[resultidx].header.clbit_labels = new_result.results[resultidx].header.clbit_labels[0:-2]
                new_result.results[resultidx].header.memory_slots = 5

                for reg_key in old_counts:
                    reg_bits = reg_key.split(' ')
                    if (reg_bits[0]==sf):#reg_key is in the form like '00 10011', reg_bts[0]='00'
                        new_counts[reg_bits[1]]=old_counts[reg_key]

                new_result.results[resultidx].data.counts = \
                    new_result.results[resultidx]. \
                    data.counts.from_dict(new_counts)

                result_sf[sf]=new_result

    #send the results to the state tomography filter
    
    tomo_qc_sim={}
    for sf in ['00','01','10','11']:
        tomo_qc_sim_sf = tom.StateTomographyFitter(result_sf[sf], qctom_no_anc, meas_basis='Pauli')
        tomo_qc_sim[sf]=tomo_qc_sim_sf
    
#     max_job_count = 5
#     max_exp_per_job = 54
#     qctom = tom.state_tomography_circuits(qc0, data_list, meas_labels='Pauli', meas_basis='Pauli')
#     """ already measured 5,6 in qctom above
#     for qc in qctom:
#         ca = ClassicalRegister(2)
#         qc.add_register(ca)
#         qc.measure([q0[5],q0[6]],[ca[0],ca[1]])
#     """
#     qctom = transpile(qctom, backend=backend)
    
#     #results_real=[]
#     #tomo_qc_real=[]
    
#     job_manager = IBMQJobManager()
#     start_time = time.time()
#     #for qc in qctom:
#     job_set_foo = job_manager.run(qctom, backend=backend, name='qctom', shots=2048, max_experiments_per_job=54)
#     job_status = job_set_foo.statuses()
#     while job_status[max_job_count-1] not in JOB_FINAL_STATES:
#         print(f'Status @ {time.time()-start_time:0.0f} s: {job_status[max_job_count-1]},'
#               f' ')
#         time.sleep(10)
#         job_status = job_set_foo.statuses()

#     results_real = job_set_foo.results()   
#     tomo_qc_real = tom.StateTomographyFitter(results_real, qctom, meas_basis='Pauli')
#     #results_real.append(results_0)
#     #tomo_qc_real.append(tomo_qc)
    results_real = tomo_qc_real = 0
    return (psi_qc, results_sim, result_sf, tomo_qc_sim, results_real, tomo_qc_real)

In [11]:
dict={}
for i in range(3):
    data = executeStatePreparation(i)
    dict[i]=data
    #pickle.dump(data, open("Stabilizer%d"%i, "wb"))


### Analyze the first stabilizer's result

In [12]:
psi_qc0, results_sim0, result_sf0, tomo_qc_sim0, results_real0, tomo_qc_real0=dict[0]

In [ ]:
# qc0=StatePreparation(0)
# qc0.draw()
# qc1=deepcopy(qc0)
# ca1 = ClassicalRegister(2)
# qc1.add_register(ca1)
# qc1.measure([5,6],[0,1])
# job0 = execute(qc1, backend_sv)

In [ ]:
# job0.result()

In [ ]:
# reduced_sv = reduced(job0.result().get_statevector(),'00')
# print(reduced_sv)

In [13]:
psi_qc0

array([ 0.1767767-1.08244507e-17j,  0.1767767-1.08244507e-17j,
        0.1767767-1.08244507e-17j, -0.1767767+1.08244507e-17j,
        0.1767767-1.08244507e-17j,  0.1767767-1.08244507e-17j,
       -0.1767767+1.08244507e-17j,  0.1767767-1.08244507e-17j,
        0.1767767-1.08244507e-17j,  0.1767767-1.08244507e-17j,
        0.1767767-1.08244507e-17j, -0.1767767+1.08244507e-17j,
       -0.1767767+1.08244507e-17j, -0.1767767+1.08244507e-17j,
        0.1767767-1.08244507e-17j, -0.1767767+1.08244507e-17j,
        0.1767767-1.08244507e-17j, -0.1767767+1.08244507e-17j,
        0.1767767-1.08244507e-17j,  0.1767767-1.08244507e-17j,
        0.1767767-1.08244507e-17j, -0.1767767+1.08244507e-17j,
       -0.1767767+1.08244507e-17j, -0.1767767+1.08244507e-17j,
       -0.1767767+1.08244507e-17j,  0.1767767-1.08244507e-17j,
       -0.1767767+1.08244507e-17j, -0.1767767+1.08244507e-17j,
        0.1767767-1.08244507e-17j, -0.1767767+1.08244507e-17j,
       -0.1767767+1.08244507e-17j, -0.1767767+1.0824450

In [14]:
reduced_sv = reduced(psi_qc0,'00')
print(reduced_sv)

[ 0.1767767-1.08244507e-17j  0.1767767-1.08244507e-17j
  0.1767767-1.08244507e-17j -0.1767767+1.08244507e-17j
  0.1767767-1.08244507e-17j  0.1767767-1.08244507e-17j
 -0.1767767+1.08244507e-17j  0.1767767-1.08244507e-17j
  0.1767767-1.08244507e-17j  0.1767767-1.08244507e-17j
  0.1767767-1.08244507e-17j -0.1767767+1.08244507e-17j
 -0.1767767+1.08244507e-17j -0.1767767+1.08244507e-17j
  0.1767767-1.08244507e-17j -0.1767767+1.08244507e-17j
  0.1767767-1.08244507e-17j -0.1767767+1.08244507e-17j
  0.1767767-1.08244507e-17j  0.1767767-1.08244507e-17j
  0.1767767-1.08244507e-17j -0.1767767+1.08244507e-17j
 -0.1767767+1.08244507e-17j -0.1767767+1.08244507e-17j
 -0.1767767+1.08244507e-17j  0.1767767-1.08244507e-17j
 -0.1767767+1.08244507e-17j -0.1767767+1.08244507e-17j
  0.1767767-1.08244507e-17j -0.1767767+1.08244507e-17j
 -0.1767767+1.08244507e-17j -0.1767767+1.08244507e-17j]


Since the simulation is ideal, syndrome and flag are always 0

In [15]:
rho_qc_sf0 = (tomo_qc_sim0['00'].fit(method='lstsq'))
# print(tomo_qc_sim0['01'])
# print(result_sf0)
# rho_qc_sf[1] = (tomo_qc_sim0['01'].fit(method='lstsq'))
# rho_qc_sf[2] = (tomo_qc_sim0['10'].fit(method='lstsq'))
# rho_qc_sf[3] = (tomo_qc_sim0['11'].fit(method='lstsq'))

fidelity between the reduced statevector of the circuit without measurement, and the reconstructed state of simulation tomography results

In [16]:
F_qc_sf0 = state_fidelity(reduced(psi_qc0,'00'), rho_qc_sf0)
print(F_qc_sf0)

0.9928004856689024


The fidelity seems rather poor :(.(Not any more. :D) And the two density matrices, represented below, are very different(not so different).

In [17]:
psi_qc0_vec=reduced(psi_qc0, '00').reshape(32,1)

In [18]:
dm_qc0=np.dot(psi_qc0_vec, psi_qc0_vec.conj().T)
print(dm_qc0)

[[ 0.03125+1.25596969e-35j  0.03125+1.25596969e-35j
   0.03125+1.25596969e-35j ... -0.03125-1.25596969e-35j
  -0.03125-1.25596969e-35j -0.03125-1.25596969e-35j]
 [ 0.03125+1.25596969e-35j  0.03125+1.25596969e-35j
   0.03125+1.25596969e-35j ... -0.03125-1.25596969e-35j
  -0.03125-1.25596969e-35j -0.03125-1.25596969e-35j]
 [ 0.03125+1.25596969e-35j  0.03125+1.25596969e-35j
   0.03125+1.25596969e-35j ... -0.03125-1.25596969e-35j
  -0.03125-1.25596969e-35j -0.03125-1.25596969e-35j]
 ...
 [-0.03125-1.25596969e-35j -0.03125-1.25596969e-35j
  -0.03125-1.25596969e-35j ...  0.03125+1.25596969e-35j
   0.03125+1.25596969e-35j  0.03125+1.25596969e-35j]
 [-0.03125-1.25596969e-35j -0.03125-1.25596969e-35j
  -0.03125-1.25596969e-35j ...  0.03125+1.25596969e-35j
   0.03125+1.25596969e-35j  0.03125+1.25596969e-35j]
 [-0.03125-1.25596969e-35j -0.03125-1.25596969e-35j
  -0.03125-1.25596969e-35j ...  0.03125+1.25596969e-35j
   0.03125+1.25596969e-35j  0.03125+1.25596969e-35j]]


In [19]:
print(np.trace(rho_qc_sf0))
print(rho_qc_sf0)

(1+0j)
[[ 0.03136445+0.00000000e+00j  0.03117663-1.67919260e-04j
   0.03074572-1.46484788e-04j ... -0.03119449+3.89497741e-04j
  -0.03122979+3.93402216e-04j -0.03109915+2.09946250e-04j]
 [ 0.03117663+1.67919260e-04j  0.03171004+0.00000000e+00j
   0.03098451-3.36959570e-05j ... -0.03124828+3.01299815e-04j
  -0.03122727+8.60883794e-07j -0.03142932-9.76911175e-05j]
 [ 0.03074572+1.46484788e-04j  0.03098451+3.36959570e-05j
   0.03054031+0.00000000e+00j ... -0.0307968 +3.76631029e-04j
  -0.0306861 +1.17572007e-04j -0.03071857+1.89755318e-05j]
 ...
 [-0.03119449-3.89497741e-04j -0.03124828-3.01299815e-04j
  -0.0307968 -3.76631029e-04j ...  0.03137527+0.00000000e+00j
   0.03114943+1.26084151e-05j  0.0311935 +2.73194293e-04j]
 [-0.03122979-3.93402216e-04j -0.03122727-8.60883794e-07j
  -0.0306861 -1.17572007e-04j ...  0.03114943-1.26084151e-05j
   0.03126838+0.00000000e+00j  0.03116328+1.10161310e-04j]
 [-0.03109915-2.09946250e-04j -0.03142932+9.76911175e-05j
  -0.03071857-1.89755318e-05j ...  

The following fidelity calculations are incorrect because noisy experimental results include those with non-zero syndrome or flag.

The fidelity between reconstruction using the simulation data and reconstruction using the real qc data

In [ ]:
rho_real_sf0= tomo_qc_real0.fit(method='lstsq')

In [ ]:
F_qc_0=state_fidelity(rho_qc_sf0, rho_real_sf0)
print(F_qc_0)

Fidelity between the real-quantum-computer-data reconstruction and the ideal statevector

In [ ]:
F_qc_1=state_fidelity(reduced(psi_qc0,'00'), rho_real_sf0)
print(F_qc_1)

In [ ]:
results_real0.get_counts(1)